# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [1]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer
 


C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Global Baseline Config

In [2]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="paper_barlowtwins"   

global_config["prjhead_dim"]=[2048,2048]    
global_config["predhead_dim"]=[]
global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="LARS"    
global_config["schedule"]="LambdaLR"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=32

# Define global view function
global_SSL_augmentation = {
    'RandomResizedCrop': {'size': global_config["input_size"], "scale":(0.08,1.0)},
    'RandomApply':{'transforms':[RandomRotation(degrees=90)], 'p':0},
    'RandomHorizontalFlip': {'p': 0.5},
    'RandomVerticalFlip':  {'p':0},
    'RandomApply':{'transforms': [ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4,hue=0.1)], 'p':0.8},
    'RandomGrayscale' :{'p':0.2},
    #'RandomSolarize':{'threshold':128, 'p':0.1},
    'RandomApply':{'transforms':[GaussianBlur(kernel_size=3,sigma=(0.1, 2))],'p':0},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}

test_SSL_augmentation = {
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
} 

# THIS IS THE CODE TO Monitor the KNN accuracy for each epoch
p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=dict2transformer(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets

In [ ]:
pipe_model(config=global_config,MonitoringbyKNN=p_knndata) # All save the validation

# Grid Run Config

## Config1: Baseline of Cifar with Batch 512

In [ ]:

for baseline in [1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_Baseline_NNNNN"
    config["batch_size"]=512
    config["max_epochs"]=200
    

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\paper_barlowtwins\BT_Baseline_NNNNN exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                      | Params
--------------------------------

Epoch 0:  11%|███████▎                                                        | 11/97 [00:08<01:03,  1.35it/s, v_num=0]

## Config2: Different batch vs FastSim

In [ ]:
for batch_num in [16,32,64,128,256,512,1024,2048,4096]:
#for batch_num in [256]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"Toymodel_all_batch_{batch_num}"
    config["views"]=3
    config["views_mode"]="all"  # overZ #overLoss #overBoth
    config["model"]="Toymodel"
    config["predhead"]=True
    config["batch_size"]=batch_num
    config["max_epochs"]=5
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    pmodel=pipe_model(config=config) 
    #pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",precision='16-mixed') #,check_val_every_n_epoch =5
    trainer.fit(pmodel, pdata.dataloader)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
    

## VICREG

In [ ]:

for batch_num in [32, 64,128,256,512,1024,2048]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"barlow_batch_{batch_num}"
    config["batch"]=batch_num
    config["model"]="BarlowTwins"

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    # THIS IS THE CODE TO LOAD MODEL
    pmodel=pipe_model(config=config)

    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start")
    trainer.fit(pmodel, pdata.dataloader)  

    del pdata
    del pmodel
    del trainer
    # Use this if you want to CONTINUE to train
    #trainer1=Trainer(config, model_mode="continue", extra_epoch=0)
    #trainer1.fit(pmodel, pdata.dataloader,ckpt_path="latest")  

## Config more ... 

In [ ]:

for batch_num in [32, 64,128,256,512,1024,2048]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"barlow_batch_{batch_num}"
    config["batch"]=batch_num
    config["model"]="BarlowTwins"

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    # THIS IS THE CODE TO LOAD MODEL
    pmodel=pipe_model(config=config)

    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start")
    trainer.fit(pmodel, pdata.dataloader)  

    del pdata
    del pmodel
    del trainer
    # Use this if you want to CONTINUE to train
    #trainer1=Trainer(config, model_mode="continue", extra_epoch=0)
    #trainer1.fit(pmodel, pdata.dataloader,ckpt_path="latest")  

# Evaluation

In [ ]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/views VS model/", reg="Toymodel_all_batch_[0-9]+")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=dict2transformer(test_augmentation,view=1))


In [ ]:
aaa=eval_linear(pdata, models=collate, device=global_config["device"])


In [ ]:
aaa